In [2]:
!source activate py36

In [3]:
import sys

sys.path.append('/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/home/faske/g/ipynb/include_utils")

import math
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [4]:
analysis_dir = '/home/faske/g/rabbit/subset/angsd'

In [5]:
cd $analysis_dir

/working/tfaske/rabbit/subset/angsd


# Calculating nt diversity (pi) 
must split vcf by each population

In [6]:
Pop_ID_df = pd.read_csv(os.path.join(analysis_dir, "Pop_ID_Loc_Var.csv"), sep=",")
Pop_ID_df.head()

,Pop,ID,Name,Lat,Long,State,Species,Variety,Color,Notes
0,AS,16,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN
1,AS,17,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN
2,AS,18,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN
3,AS,19,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN
4,AS,20,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN


In [7]:
Bam = []
for i in range(0,len(Pop_ID_df)):
    name = ["/home/faske/g/rabbit/subset/bowtie2/EN_",Pop_ID_df.Pop.loc[i],"_",str(Pop_ID_df.ID.loc[i]),".fastq.gz_sorted.bam"]
    #print(name)
    Bam.append("".join(name))
Pop_ID_df['Bam'] = Bam
Pop_ID_df.head()
#print(len(Bam),len(Pop_ID_df))

,Pop,ID,Name,Lat,Long,State,Species,Variety,Color,Notes,Bam
0,AS,16,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN,/home/faske/g/rabbit/subset/bowtie2/EN_AS_16.f...
1,AS,17,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN,/home/faske/g/rabbit/subset/bowtie2/EN_AS_17.f...
2,AS,18,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN,/home/faske/g/rabbit/subset/bowtie2/EN_AS_18.f...
3,AS,19,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN,/home/faske/g/rabbit/subset/bowtie2/EN_AS_19.f...
4,AS,20,Austin Summit,39.473104,117.049449,NV,NaN,hololeucus,gray,NaN,/home/faske/g/rabbit/subset/bowtie2/EN_AS_20.f...


In [8]:
pops = Pop_ID_df.Pop.unique()
for i in range(0,len(pops)):
    pop = str(pops[i])
    names = Pop_ID_df[(Pop_ID_df.Pop == pop)].Bam
    #print(names)
    pop_file = analysis_dir + "/"+ pop + "_bam_name.txt"
    #print(pop_file)
    names.to_csv(path=pop_file,sep='\n',header=False,index=False)

/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'path' will be renamed to 'path_or_buf'.
  


# Get sfs.idx #

In [9]:
!source activate py34

In [10]:
pwd

'/working/tfaske/rabbit/subset/angsd'

In [11]:
assembly = '/home/faske/g/rabbit/assembly/bowtie_6680/reference.fasta'

In [ ]:
pops = Pop_ID_df.Pop.unique()
for i in range(0,len(pops)):
    pop = str(pops[i])
    bam_file = !ls {analysis_dir}/{pop}_bam_name.txt
    !angsd -bam $bam_file -doSaf 1 -anc $assembly -GL 1 -P 6 -out $pop &

In [18]:
with open("./run_angsd.sh", "w") as o:
    o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
source activate py34\n""")
    pops = Pop_ID_df.Pop.unique()
    ncores = 6
    for i in range(0,len(pops)):
        pop = str(pops[i])
        bam_file = os.path.join(analysis_dir,''.join([pop,'_bam_name.txt']))
        o.write("angsd -bam %s -doSaf 1 -anc %s -GL 1 -P %s -out %s & \ndisown -h\n\n" % (bam_file,assembly,ncores,pop))
        
      

# in terminal #
cd /home/faske/g/rabbit/subset/angsd </br>
bash run_angsd.sh > angsd.out & </br>
disown -h</br>

# Make real sfs

In [8]:
Pop_ID_df = pd.read_csv(os.path.join(analysis_dir, "Pop_ID_Loc_Var.csv"), sep=",")

In [10]:
with open("./run_realSFS.sh", "w") as o:
    o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
source activate py34\n""")
    pops = Pop_ID_df.Pop.unique()
    ncores = 6
    for i in range(0,len(pops)):
        pop = str(pops[i])
        o.write("realSFS %s.saf.idx -P %s > %s.sfs & \ndisown -h\n\n" % (pop,ncores,pop))
        
#cd /home/faske/g/rabbit/subset/angsd 
#bash run_realSFS.sh > realSFS.out & disown -h     

# do theta

In [11]:
Pop_ID_df = pd.read_csv(os.path.join(analysis_dir, "Pop_ID_Loc_Var.csv"), sep=",")

In [12]:
assembly = '/home/faske/g/rabbit/assembly/bowtie_6680/reference.fasta'

In [14]:
with open("./run_doTheta.sh", "w") as o:
    o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
source activate py34\n""")
    pops = Pop_ID_df.Pop.unique()
    for i in range(0,len(pops)):
        pop = str(pops[i])
        o.write("angsd -bam %s_bam_name.txt -out %s -doThetas 1 -doSaf 1 -pest %s.sfs -anc %s -GL 1 & \ndisown -h\n\n" % (pop,pop,pop,assembly))
        
#cd /home/faske/g/rabbit/subset/angsd 
#bash run_doTheta.sh > doTheta.out & disown -h 


# do stat

In [12]:
with open("./run_doStat.sh", "w") as o:
    o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
source activate py34\n""")
    pops = Pop_ID_df.Pop.unique()
    for i in range(0,len(pops)):
        pop = str(pops[i])
        o.write("thetaStat do_stat %s.thetas.idx -win 1 -step 1 & \ndisown -h\n\n" % (pop))
        
#cd /home/faske/g/rabbit/subset/angsd 
#bash run_doStat.sh > doStat.out & 
#disown -h 


# theta_out

In [16]:
with open("./run_thetaOut.sh", "w") as o:
    o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
source activate py34\n""")
    pops = Pop_ID_df.Pop.unique()
    for i in range(0,len(pops)):
        pop = str(pops[i])
        o.write("thetaStat print %s.thetas.idx > %s.theta_out & \ndisown -h\n\n" % (pop,pop))
        
#cd /home/faske/g/rabbit/subset/angsd 
#bash run_thetaOut.sh > thetaOut.out & disown -h 


# OLD PI with VCF

In [10]:
def get_pi(filedir, prefix):
    pi_file = !ls {filedir}/{prefix}.sites.pi
    pi_df = pd.read_csv(pi_file[0], sep="\t")
    pi = pi_df.PI.mean()
    #print(pi)
    return pi

In [11]:
pi_array=[]
pops = Pop_ID_df.Pop.unique()
for i in range(0,len(pops)):
    pop = pops[i]
    pi = get_pi(analysis_dir,pop)
    pi_array.append(pi)
pop_pi = {'Pop':pops,'pi':pi_array}
pop_pi_df = pd.DataFrame(pop_pi)
pop_pi_df.head()

,Pop,pi
0,AS,0.264486
1,BM,0.278885
2,BV,0.277280
3,DC,0.272952
4,DH,0.275095


In [12]:
pop_pi_file = os.path.join(analysis_dir,'pop_pi.csv')
pop_pi_df.to_csv(path_or_buf=pop_pi_file,index=False)

# test

In [19]:
SS_theta = pd.read_csv(os.path.join(analysis_dir, "SS.theta_out"), sep="\t")

In [20]:
SS_theta.head()

,#Chromo,Pos,Watterson,Pairwise,thetaSingleton,thetaH,thetaL
0,dDocent_Contig_6,2,-inf,-72.711815,-102.585783,-74.791252,-73.287178
1,dDocent_Contig_6,3,-inf,-72.103607,-103.278930,-74.183044,-72.678970
2,dDocent_Contig_6,4,-inf,-71.733322,-103.278930,-73.812759,-72.308685
3,dDocent_Contig_6,5,-inf,-72.286049,-103.278930,-74.365486,-72.861412
4,dDocent_Contig_6,6,-inf,-72.545708,-103.278930,-74.625145,-73.121071


In [36]:
np.mean(np.exp(SS_theta.Pairwise))

0.019512152172408606

In [37]:
josh = pd.read_csv('/working/jahner/desert_bigQ/sam_sai/DB_254_angsd.theta_out',sep='\t')

In [40]:
np.mean(np.exp(josh.Pairwise))

0.0007926749313000031